In [ ]:
!pip install -q streamlit
!pip install -q streamlit_cropper
!npm install localtunnel
!git clone https://github.com/quangbinh113/CV20222_Adversarial_Attack.git
!git clone https://github.com/ultralytics/yolov5.git
!pip install gdown
!pip install ultralytics
# %cd yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.0/953.0 kB 8.6 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm

In [ ]:
!gdown 1b10veNEl4LDnjsaODdsa90D2r4ztoT9h
!gdown 1FgVozWlrhPHoPQ47pW4cUXEBnezU7QzL

Downloading...
From: https://drive.google.com/uc?id=1b10veNEl4LDnjsaODdsa90D2r4ztoT9h
To: /content/best_v1.pt
100% 14.4M/14.4M [00:00<00:00, 106MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1FgVozWlrhPHoPQ47pW4cUXEBnezU7QzL
To: /content/best_noise.pt
100% 56.7M/56.7M [00:00<00:00, 90.4MB/s]


In [ ]:
import os
os.makedirs('data', exist_ok = True)

In [ ]:
%%writefile /content/app.py

import streamlit as st
import numpy as np
from streamlit_cropper import st_cropper
from PIL import Image
import cv2
import shutil
import os

st.set_option('deprecation.showfileUploaderEncoding', False)

# Upload an image and set some options for demo purposes
st.header("Face Defender")
img_file = st.sidebar.file_uploader(label = 'Upload a file', type = ['png', 'jpg'])
realtime_update = st.sidebar.checkbox(label = "Update in Real Time", value=True)
box_color = st.sidebar.color_picker(label = "Box Color", value='#0000FF')

# aspect_choice = st.sidebar.radio(label="Aspect Ratio", options=["1:1", "16:9", "4:3", "2:3", "Free"])
# aspect_dict = {
#     "1:1": (1, 1),
#     "16:9": (16, 9),
#     "4:3": (4, 3),
#     "2:3": (2, 3),
#     "Free": None
# }
# aspect_ratio = aspect_dict[aspect_choice]

# return_type_choice = st.sidebar.radio(label = "Return type", options = ["Cropped image", "Rect coords"])
# return_type_dict = {
#     "Cropped image": "image",
#     "Rect coords": "box"
# }
# return_type = return_type_dict[return_type_choice]

if 'count' not in st.session_state:
  st.session_state['count'] = 1

if img_file:
    img = Image.open(img_file)
    # for i in range(3):
    cols_1 = st.columns(2)
    img.save('/content/data/00002.jpg')
    os.system('python /content/yolov5/detect.py --source /content/data/00002.jpg --weights /content/best_v1.pt')
    cols_1[0].image(img, caption = 'Before attack')
    if st.session_state.count == 1:
      cols_1[1].image(Image.open(f'/content/yolov5/runs/detect/exp/00002.jpg'), caption = 'Detection result')
    else:
      cols_1[1].image(Image.open(f'/content/yolov5/runs/detect/exp{st.session_state.count}/00002.jpg'), 'Detection result')
    st.session_state.count += 1
    os.system('python /content/CV20222_Adversarial_Attack/AttacksAdversarial/inference.py')

    img_noise = Image.open('/content/data/attack_image/0.jpg')
    cols_2 = st.columns(2)
    cols_2[0].image(img_noise, caption = 'After attack')
    os.system('python /content/yolov5/detect.py --source /content/data/attack_image/0.jpg --weights /content/best_v1.pt')
    cols_2[1].image(Image.open(f'/content/yolov5/runs/detect/exp{st.session_state.count}/0.jpg'), 'Detection result')
    st.session_state.count += 1

    cols_2[0].image(img_noise, caption = 'After defense by adversarial training')
    os.system('python /content/yolov5/detect.py --source /content/data/attack_image/0.jpg --weights /content/best_noise.pt')
    cols_2[1].image(Image.open(f'/content/yolov5/runs/detect/exp{st.session_state.count}/0.jpg'), 'Detection result')
    st.session_state.count += 1


    os.system('python /content/CV20222_Adversarial_Attack/AdversarialDefense/src/DenoiseSRGAN/test_only.py --device cpu --n_blocks 5 --batch_size 64 --num_workers 0 --save_dir /content/ --input_dir /content/data/00002.jpg --model_dir /content/CV20222_Adversarial_Attack/AdversarialDefense/models/Generator.pth')
    cols_3 = st.columns(2)
    img_denoised = Image.open('/content/denoised.png')
    cols_3[0] = st.image(img_denoised, 'After defense by adversarial training (SRGAN)')
    os.system('python /content/yolov5/detect.py --source /content/denoised.png --weights /content/best_noise.pt')
    cols_3[1].image(Image.open(f'/content/yolov5/runs/detect/exp{st.session_state.count}/denoised.png'), 'Detection result')
    st.session_state.count += 1

    shutil.rmtree('/content/data/attack_image')
    # img = Image.open(img_file)
    # if not realtime_update:
    #     st.write("Double click to save crop")
    # if return_type == 'box':
    #     rect = st_cropper(
    #         img,
    #         realtime_update=realtime_update,
    #         box_color=box_color,
    #         aspect_ratio=aspect_ratio,
    #         return_type=return_type
    #     )
    #     raw_image = np.asarray(img).astype('uint8')
    #     left, top, width, height = tuple(map(int, rect.values()))
    #     st.write(rect)
    #     masked_image = np.zeros(raw_image.shape, dtype='uint8')
    #     masked_image[top:top + height, left:left + width] = raw_image[top:top + height, left:left + width]
    #     st.image(Image.fromarray(masked_image), caption='masked image')
    # else:
    #     # Get a cropped image from the frontend
    #     cropped_img = st_cropper(
    #         img,
    #         realtime_update=realtime_update,
    #         box_color=box_color,
    #         aspect_ratio=aspect_ratio,
    #         return_type=return_type
    #     )

    #     # Manipulate cropped image at will
    #     st.write("Preview")
    #     _ = cropped_img.thumbnail((150, 150))
    #     st.image(cropped_img)

Overwriting /content/app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.91.255.88
npx: installed 22 in 4.394s
your url is: https://rare-experts-drive.loca.lt


In [ ]:
!rm -r /content/yolov5/runs/detect

In [ ]:
os.system('python /content/CV20222_Adversarial_Attack/AttacksAdversarial/inference.py')

0

In [ ]:
!python /content/CV20222_Adversarial_Attack/AttacksAdversarial/inference.py

Testing Image: 00002.jpg	(1/1)
0/1 attacks succeeded with epsilon=16. Continuing...
0/1 attacks succeeded with epsilon=32. Continuing...
0/1 attacks succeeded with epsilon=48. Continuing...
0/1 attacks succeeded with epsilon=64. Continuing...
0/1 attacks succeeded with epsilon=80. Continuing...
0/1 attacks succeeded with epsilon=96. Continuing...
0/1 attacks succeeded with epsilon=112. Continuing...
0/1 attacks succeeded with epsilon=128. Continuing...
0/1 attacks succeeded with epsilon=144. Continuing...
0/1 attacks succeeded with epsilon=160. Continuing...
Attack succeeded at least 80.0% of the time with epsilon: 176
/content/CV20222_Adversarial_Attack/AttacksAdversarial/inference.py:109: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_avg += ssim(img,


In [ ]:
!python /content/CV20222_Adversarial_Attack/AdversarialDefense/src/DenoiseSRGAN/test_only.py --device cpu --n_blocks 5 --batch_size 64 --num_workers 0 --save_dir /content/ --input_dir /content/data/00002.jpg --model_dir /content/CV20222_Adversarial_Attack/models/Generator.pth